In [57]:
import yfinance as yf
import pandas as pd

# Define list of tickers and timeframes
tickers = ["AC.PA", "AI.PA", "AIR.PA", "ALO.PA", "ATO.PA", "CS.PA", "BB.PA", "EN.PA", "BNP.PA", "CAP.PA", "CA.PA", "CO.PA", "ACA.PA", "BN.PA", "DSY.PA", "ENGI.PA", "EL.PA", "RF.PA", "ENX.PA", "GFC.PA", "GET.PA", "RMS.PA", "DEC.PA", "KER.PA", "OR.PA", "LR.PA", "MC.PA", "ML.PA", "ORA.PA", "RI.PA", "PUB.PA", "RNO.PA", "SGO.PA", "SAN.PA", "SU.PA", "GLE.PA", "SW.PA", "TEP.PA", "TE.PA", "HO.PA", "TTE.PA", "URW.PA", "VIE.PA", "DG.PA", "VIV.PA", "WLN.PA", "ATE.PA", "AMUN.PA", "BOL.PA", "CARM.PA", "COV.PA", "DBG.PA", "EDEN.PA", "ELIOR.PA", "ERA.PA", "ETL.PA", "ADP.PA", "ICAD.PA", "NK.PA", "LI.PA", "MERY.PA", "NEX.PA", "NXI.PA", "RXL.PA", "RUI.PA", "SK.PA", "SESG.PA", "SOI.PA", "SOP.PA", "SPIE.PA", "TFI.PA", "TRI.PA", "UBI.PA", "VK.PA", "VIRP.PA", "MF.PA", "IDL.PA", "ITP.PA", "LSS.PA", "ARG.PA", "MMT.PA", "MRN.PA", "OPM.PA", "VCT.PA", "MAU.PA", "BEN.PA", "VIRI.PA", "VLA.PA", "ES.PA", "EAPI.PA", "STMPA.PA", "BIM.PA", "BVI.PA", "FGR.PA", "ERF.PA", "FR.PA", "AKE.PA", "GTT.PA", "FRVIA.PA", "RCO.PA", "COFA.PA", "AYV.PA", "AF.PA", "VU.PA", "IPN.PA", "NEOEN.PA", "VRLA.PA", "IPS.PA", "PLX.PA", "SCR.PA", "DIM.PA", "AM.PA", "ELIS.PA"]

# Timeframes in trading days
timeframes = {"1m": 21, "2m": 42, "6m": 126}

# Function to get indicators with float-only values
def calculate_rsi(data, window=14):
    delta = data['Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window).mean()
    avg_loss = loss.rolling(window).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return float(rsi.iloc[-1]) if not rsi.empty else None

def calculate_moving_average(data, window):
    return float(data['Close'].rolling(window).mean().iloc[-1]) if not data.empty else None

def calculate_bollinger_bands(data, window=20, num_std=2):
    ma = data['Close'].rolling(window).mean()
    std = data['Close'].rolling(window).std()
    upper_band = ma + num_std * std
    lower_band = ma - num_std * std
    return float(ma.iloc[-1]), float(upper_band.iloc[-1]), float(lower_band.iloc[-1]) if not data.empty else (None, None, None)

def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    short_ema = data['Close'].ewm(span=short_window, adjust=False).mean()
    long_ema = data['Close'].ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    macd_signal = macd.ewm(span=signal_window, adjust=False).mean()
    macd_hist = macd - macd_signal
    return float(macd.iloc[-1]), float(macd_signal.iloc[-1]), float(macd_hist.iloc[-1]) if not macd.empty else (None, None, None)

# Get hourly data for last week
def get_hourly_data(ticker):
    return yf.download(ticker, period="5d", interval="1h")

# Data storage
data_dict = {}

# Main processing loop
for ticker in tickers:
    print(f"Processing {ticker}...")
    stock_data = yf.download(ticker, period="6mo", interval="1d")
    hourly_data = get_hourly_data(ticker)
    current_price = float(stock_data['Close'].iloc[-1]) if not stock_data.empty else None

    # Initialize dict for current ticker
    data_dict[ticker] = {"Current_Price": current_price}

    # Calculate daily indicators
    for name, days in timeframes.items():
        recent_data = stock_data.tail(days)
        data_dict[ticker][f"RSI_{name}"] = calculate_rsi(recent_data)
        data_dict[ticker][f"SMA20_{name}"] = calculate_moving_average(recent_data, 20)
        data_dict[ticker][f"SMA50_{name}"] = calculate_moving_average(recent_data, 50)

        # Bollinger Bands and MACD
        bb_ma, bb_upper, bb_lower = calculate_bollinger_bands(recent_data)
        data_dict[ticker][f"BB_upper_{name}"] = bb_upper
        data_dict[ticker][f"BB_lower_{name}"] = bb_lower
        macd, macd_signal, macd_hist = calculate_macd(recent_data)
        data_dict[ticker][f"MACD_{name}"] = macd
        data_dict[ticker][f"MACD_signal_{name}"] = macd_signal
        data_dict[ticker][f"MACD_hist_{name}"] = macd_hist

    # Calculate hourly indicators for 1 week
    data_dict[ticker]['RSI_1w_hourly'] = calculate_rsi(hourly_data, window=14)
    data_dict[ticker]['SMA20_1w_hourly'] = calculate_moving_average(hourly_data, 20)
    data_dict[ticker]['SMA50_1w_hourly'] = calculate_moving_average(hourly_data, 50)

    bb_ma, bb_upper, bb_lower = calculate_bollinger_bands(hourly_data)
    data_dict[ticker]['BB_upper_1w_hourly'] = bb_upper
    data_dict[ticker]['BB_lower_1w_hourly'] = bb_lower

    macd, macd_signal, macd_hist = calculate_macd(hourly_data)
    data_dict[ticker]['MACD_1w_hourly'] = macd
    data_dict[ticker]['MACD_signal_1w_hourly'] = macd_signal
    data_dict[ticker]['MACD_hist_1w_hourly'] = macd_hist

# Convert to DataFrame and save to CSV
df = pd.DataFrame.from_dict(data_dict, orient="index")
df.to_csv("/content/sbf_120_indicators.csv")
print("Saved data to /content/sbf_120_indicators.csv")

Processing AC.PA...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
<ipython-input-57-b5d396ecc43b>:56: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  current_price = float(stock_data['Close'].iloc[-1]) if not stock_data.empty else None
<ipython-input-57-b5d396ecc43b>:24: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(rsi.iloc[-1]) if not rsi.empty else None
<ipython-input-57-b5d396ecc43b>:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(data['Close'].rolling(window).mean().iloc[-1]) if not data.empty else None
<ipython-input-57-b5d396ecc43b>:34: FutureWarning: Calling float on a single element Series

Processing AI.PA...


[*********************100%***********************]  1 of 1 completed
<ipython-input-57-b5d396ecc43b>:56: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  current_price = float(stock_data['Close'].iloc[-1]) if not stock_data.empty else None
<ipython-input-57-b5d396ecc43b>:24: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(rsi.iloc[-1]) if not rsi.empty else None
<ipython-input-57-b5d396ecc43b>:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(data['Close'].rolling(window).mean().iloc[-1]) if not data.empty else None
<ipython-input-57-b5d396ecc43b>:34: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(se

Processing AIR.PA...


...

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
<ipython-input-57-b5d396ecc43b>:56: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  current_price = float(stock_data['Close'].iloc[-1]) if not stock_data.empty else None
<ipython-input-57-b5d396ecc43b>:24: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(rsi.iloc[-1]) if not rsi.empty else None
<ipython-input-57-b5d396ecc43b>:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(data['Close'].rolling(window).mean().iloc[-1]) if not data.empty else None
<ipython-input-57-b5d396ecc43b>:34: FutureWarning: Calling float on a single element Series

Processing ELIS.PA...



[*********************100%***********************]  1 of 1 completed

Saved data to /content/sbf_120_indicators.csv



<ipython-input-57-b5d396ecc43b>:56: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  current_price = float(stock_data['Close'].iloc[-1]) if not stock_data.empty else None
<ipython-input-57-b5d396ecc43b>:24: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(rsi.iloc[-1]) if not rsi.empty else None
<ipython-input-57-b5d396ecc43b>:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(data['Close'].rolling(window).mean().iloc[-1]) if not data.empty else None
<ipython-input-57-b5d396ecc43b>:34: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(ma.iloc[-1]), float(upper_band.ilo

In [61]:
import pandas as pd
import numpy as np

# Company name mapping
COMPANY_NAMES = {
    'AC.PA': 'Accor',
    'AI.PA': 'Air Liquide',
    'AIR.PA': 'Airbus',
    'ALO.PA': 'Alstom',
    'ATO.PA': 'Atos',
    'CS.PA': 'AXA',
    'BB.PA': 'BIC',
    'EN.PA': 'Bouygues',
    'BNP.PA': 'BNP Paribas',
    'CAP.PA': 'Capgemini',
    'CA.PA': 'Carrefour',
    'CO.PA': 'Casino Guichard-Perrachon',
    'ACA.PA': 'Crédit Agricole',
    'BN.PA': 'Danone',
    'DSY.PA': 'Dassault Systèmes',
    'ENGI.PA': 'Engie',
    'EL.PA': 'EssilorLuxottica',
    'RF.PA': 'Eurazeo',
    'ENX.PA': 'Euronext',
    'GFC.PA': 'Gecina',
    'GET.PA': 'Getlink',
    'RMS.PA': 'Hermès International',
    'DEC.PA': 'JCDecaux',
    'KER.PA': 'Kering',
    'OR.PA': "L'Oréal",
    'LR.PA': 'Legrand',
    'MC.PA': 'LVMH',
    'ML.PA': 'Michelin',
    'ORA.PA': 'Orange',
    'RI.PA': 'Pernod Ricard',
    'PUB.PA': 'Publicis Groupe',
    'RNO.PA': 'Renault',
    'SGO.PA': 'Saint-Gobain',
    'SAN.PA': 'Sanofi',
    'SU.PA': 'Schneider Electric',
    'GLE.PA': 'Société Générale',
    'SW.PA': 'Sodexo',
    'TEP.PA': 'Teleperformance',
    'TE.PA': 'Technip Energies',
    'HO.PA': 'Thales',
    'TTE.PA': 'TotalEnergies',
    'URW.PA': 'Unibail-Rodamco-Westfield',
    'VIE.PA': 'Veolia Environnement',
    'DG.PA': 'Vinci',
    'VIV.PA': 'Vivendi',
    'WLN.PA': 'Worldline',
    'ATE.PA': 'Alten',
    'AMUN.PA': 'Amundi',
    'BOL.PA': 'Bolloré',
    'CARM.PA': 'Carmila',
    'COV.PA': 'Covivio',
    'DBG.PA': 'Derichebourg',
    'EDEN.PA': 'Edenred',
    'ELIOR.PA': 'Elior Group',
    'ERA.PA': 'Eramet',
    'ETL.PA': 'Eutelsat Communications',
    'ADP.PA': 'ADP',
    'ICAD.PA': 'Icade',
    'NK.PA': 'Imerys',
    'LI.PA': 'Klépierre',
    'MERY.PA': 'Mercialys',
    'NEX.PA': 'Nexans',
    'NXI.PA': 'Nexity',
    'RXL.PA': 'Rexel',
    'RUI.PA': 'Rubis',
    'SK.PA': 'SEB',
    'SESG.PA': 'SES',
    'SOI.PA': 'Soitec',
    'SOP.PA': 'Sopra Steria Group',
    'SPIE.PA': 'Spie',
    'TFI.PA': 'TF1',
    'TRI.PA': 'Trigano',
    'UBI.PA': 'Ubisoft Entertainment',
    'VK.PA': 'Vallourec',
    'VIRP.PA': 'Virbac',
    'MF.PA': 'Wendel',
    'IDL.PA': 'ID Logistics Group',
    'ITP.PA': 'Interparfums',
    'LSS.PA': 'Lectra',
    'ARG.PA': 'Argan',
    'MMT.PA': 'M6 - Métropole Télévision',
    'MRN.PA': 'Mersen',
    'OPM.PA': 'OPMobility',
    'VCT.PA': 'Vicat',
    'MAU.PA': 'Maurel & Prom',
    'BEN.PA': 'Beneteau',
    'VIRI.PA': 'Viridian',
    'VLA.PA': 'Valneva',
    'ES.PA': 'Esso',
    'EAPI.PA': 'Euroapi',
    'STMPA.PA': 'STMicroelectronics',
    'BIM.PA': 'bioMérieux',
    'BVI.PA': 'Bureau Veritas',
    'FGR.PA': 'Eiffage',
    'ERF.PA': 'Eurofins Scientific',
    'FR.PA': 'Valeo',
    'AKE.PA': 'Arkema',
    'GTT.PA': 'GTT',
    'FRVIA.PA': 'Forvia',
    'RCO.PA': 'Remy Cointreau',
    'COFA.PA': 'Coface',
    'AYV.PA': 'Ayvens',
    'AF.PA': 'Air France-KLM',
    'VU.PA': 'Visiativ',
    'IPN.PA': 'Ipsen',
    'NEOEN.PA': 'Neoen',
    'VRLA.PA': 'Verallia',
    'SOLB.PA': 'Solvay',
    'IPS.PA': 'Ipsos',
    'PLX.PA': 'Pluxee',
    'SCR.PA': 'SCOR',
    'DIM.PA': 'Sartorius Stedim Biotech',
    'AM.PA': 'Dassault Aviation',
    'ELIS.PA': 'Elis'
}

def analyze_technical_indicators(data):
    # Define weights for different timeframes
    TIMEFRAME_WEIGHTS = {
        "1m": 1.5,    # Most recent data gets highest weight
        "2m": 1.3,    # Medium-term data
        "6m": 1.0,    # Longer-term data
        "1w_hourly": 0.8  # Hourly data gets lowest weight due to noise
    }

    # Define weights for different indicators
    INDICATOR_WEIGHTS = {
        "RSI": 1.5,
        "Moving_Average_Crossover": 1.2,
        "Bollinger_Bands": 1.0,
        "MACD": 1.3
    }

    def analyze_rsi(rsi_value):
        if rsi_value < 30:  # Oversold
            return 1
        elif rsi_value > 70:  # Overbought
            return -1
        return 0

    def analyze_moving_averages(sma20, sma50):
        if sma20 > sma50:  # Bullish crossover
            return 1
        elif sma20 < sma50:  # Bearish crossover
            return -1
        return 0

    def analyze_bollinger_bands(price, upper_bb, lower_bb):
        if price <= lower_bb:  # Oversold
            return 1
        elif price >= upper_bb:  # Overbought
            return -1
        return 0

    def analyze_macd(macd, signal, hist):
        if macd > signal and hist > 0:  # Bullish momentum
            return 1
        elif macd < signal and hist < 0:  # Bearish momentum
            return -1
        return 0

    scores = {}

    for index, row in data.iterrows():
        ticker = row.iloc[0]  # Assuming first column is ticker
        total_score = 0
        total_weight = 0

        # Analyze each timeframe
        for timeframe in ["1m", "2m", "6m", "1w_hourly"]:
            timeframe_score = 0

            # RSI Analysis
            rsi_col = f"RSI_{timeframe}"
            if rsi_col in row.index:
                rsi_score = analyze_rsi(row[rsi_col])
                timeframe_score += rsi_score * INDICATOR_WEIGHTS["RSI"]

            # Moving Average Analysis
            sma20_col = f"SMA20_{timeframe}"
            sma50_col = f"SMA50_{timeframe}"
            if sma20_col in row.index and sma50_col in row.index:
                ma_score = analyze_moving_averages(row[sma20_col], row[sma50_col])
                timeframe_score += ma_score * INDICATOR_WEIGHTS["Moving_Average_Crossover"]

            # Bollinger Bands Analysis
            bb_upper_col = f"BB_upper_{timeframe}"
            bb_lower_col = f"BB_lower_{timeframe}"
            if bb_upper_col in row.index and bb_lower_col in row.index:
                bb_score = analyze_bollinger_bands(row["Current_Price"],
                                                 row[bb_upper_col],
                                                 row[bb_lower_col])
                timeframe_score += bb_score * INDICATOR_WEIGHTS["Bollinger_Bands"]

            # MACD Analysis
            macd_col = f"MACD_{timeframe}"
            signal_col = f"MACD_signal_{timeframe}"
            hist_col = f"MACD_hist_{timeframe}"
            if all(col in row.index for col in [macd_col, signal_col, hist_col]):
                macd_score = analyze_macd(row[macd_col],
                                        row[signal_col],
                                        row[hist_col])
                timeframe_score += macd_score * INDICATOR_WEIGHTS["MACD"]

            # Add weighted timeframe score to total
            total_score += timeframe_score * TIMEFRAME_WEIGHTS[timeframe]
            total_weight += sum(INDICATOR_WEIGHTS.values()) * TIMEFRAME_WEIGHTS[timeframe]

        # Normalize the final score to be between -100 and 100
        if total_weight > 0:
            normalized_score = (total_score / total_weight) * 100
            scores[ticker] = round(normalized_score, 2)
        else:
            scores[ticker] = 0

    return scores

def main():
    # Read the data
    data = pd.read_csv('/content/sbf_120_indicators.csv')

    # Calculate scores
    scores = analyze_technical_indicators(data)

    # Convert scores to DataFrame and add company names
    scores_df = pd.DataFrame.from_dict(scores, orient='index', columns=['Score'])
    scores_df['Company'] = scores_df.index.map(lambda x: COMPANY_NAMES.get(x, x))

    # Reorder columns to show Company first
    scores_df = scores_df[['Company', 'Score']]

    # Sort by score
    scores_df = scores_df.sort_values('Score', ascending=False)

    # Add score interpretation
    def interpret_score(score):
        if score >= 70:
            return "Strong Buy"
        elif score >= 30:
            return "Buy"
        elif score > -30:
            return "Neutral"
        elif score > -70:
            return "Sell"
        else:
            return "Strong Sell"

    scores_df['Recommendation'] = scores_df['Score'].apply(interpret_score)

    return scores_df

if __name__ == "__main__":
    result_df = main()
    print("\nLes 10 titres les plus haussiers :")
    print(result_df.head(10))
    print("\nLes 10 actions les plus baissières :")
    print(result_df.tail(10))


Les 10 titres les plus haussiers :
                      Company  Score Recommendation
VK.PA               Vallourec  31.22            Buy
RMS.PA   Hermès International  31.22            Buy
DBG.PA           Derichebourg  27.74        Neutral
AMUN.PA                Amundi  26.00        Neutral
EL.PA        EssilorLuxottica  25.65        Neutral
TEP.PA        Teleperformance  20.78        Neutral
ORA.PA                 Orange  20.78        Neutral
MC.PA                    LVMH  20.78        Neutral
VLA.PA                Valneva  20.78        Neutral
ITP.PA           Interparfums  20.78        Neutral

Les 10 actions les plus baissières :
                      Company  Score Recommendation
CAP.PA              Capgemini -22.17        Neutral
ARG.PA                  Argan -22.17        Neutral
TFI.PA                    TF1 -25.65        Neutral
DG.PA                   Vinci -27.39        Neutral
GET.PA                Getlink -27.39        Neutral
VIE.PA   Veolia Environnement -27.39      